# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [153]:
# Dependencies
import gmaps
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
from citipy import citipy
import time
import os
from datetime import datetime

from config import gkey
gmaps.configure(api_key=gkey)
from config import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [154]:
weather_csv = "../output_data/weather_df.csv"
weather_df = pd.read_csv(weather_csv)
weather_df.head()

,Unnamed: 0,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,shimoda,JP,34.67,138.95,73.00,96,89,9.26,1599419377
1,1,georgetown,MY,5.41,100.34,80.60,83,20,1.01,1599419120
2,2,albany,US,42.60,-73.97,78.01,40,18,1.16,1599419378
3,3,sao filipe,IN,20.51,76.21,77.41,80,4,1.97,1599419378
4,4,jamestown,US,42.10,-79.24,73.00,52,1,2.13,1599419079


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [155]:
# store values
locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"].astype(float)

In [156]:
# plot heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=1.55,center=(15,50))
# create layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

#add layer to map
fig.add_layer(heat_layer)

#display
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [157]:
ideal_df = weather_df.loc[(weather_df["Max Temp"] > 65) & (weather_df['Max Temp'] < 77) & (weather_df["Cloudiness"] == 0) & (weather_df['Humidity'] < 50) &(weather_df['Wind Speed'] < 15), :]

In [158]:
ideal_df = ideal_df.dropna(how = 'any')

In [159]:
ideal_df.reset_index(inplace=True)
ideal_df.head()

,index,Unnamed: 0,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,153,153,kalabo,ZM,-14.97,22.68,69.80,23,0,5.30,1599419416
1,435,435,zambezi,ZM,-13.54,23.10,71.58,22,0,8.95,1599419484
2,446,446,baherden,TM,38.44,57.43,76.46,18,0,9.66,1599419486
3,494,494,birjand,IR,32.87,59.22,69.80,19,0,6.93,1599419499
4,497,497,katherine,AU,-14.47,132.27,66.20,45,0,5.82,1599419499


In [160]:
print(ideal_df['Humidity'].max())

45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [161]:
#create df to hold info
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,Unnamed: 0,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,153,153,kalabo,ZM,-14.97,22.68,69.80,23,0,5.30,1599419416,
1,435,435,zambezi,ZM,-13.54,23.10,71.58,22,0,8.95,1599419484,
2,446,446,baherden,TM,38.44,57.43,76.46,18,0,9.66,1599419486,
3,494,494,birjand,IR,32.87,59.22,69.80,19,0,6.93,1599419499,
4,497,497,katherine,AU,-14.47,132.27,66.20,45,0,5.82,1599419499,


In [162]:
# parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": gkey
}

for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_loc = (requests.get(base_url, params=params)).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_loc["results"][0]["name"]
    except (KeyError, IndexError):
        print("No hotels here...skipping city")

No hotels here...skipping city


In [163]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_df.head()

,index,Unnamed: 0,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,153,153,kalabo,ZM,-14.97,22.68,69.80,23,0,5.30,1599419416,Sikakubete Guest House Kalabo Town Council
1,435,435,zambezi,ZM,-13.54,23.10,71.58,22,0,8.95,1599419484,Royal Kutachika Lodge
2,446,446,baherden,TM,38.44,57.43,76.46,18,0,9.66,1599419486,
3,494,494,birjand,IR,32.87,59.22,69.80,19,0,6.93,1599419499,Ghaem Hotel
4,497,497,katherine,AU,-14.47,132.27,66.20,45,0,5.82,1599419499,Beagle Motor Inn


In [164]:
humidity_locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"].astype(float)

# Add marker layer ontop of heat map
# plot heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=1.55,center=(15,50))
# create hotel marker layer
hotel_layer = gmaps.marker_layer(locations, info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

#add hotel layer to map
fig.add_layer(hotel_layer)

#create heat layer
heat_layer = gmaps.heatmap_layer(humidity_locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

#add heat layer
fig.add_layer(heat_layer)

#display
fig

IndexError: list index out of range